# Device Managment - Scripts

    Author: Jing Nghik
    Last Updated 2/9/2021

Purpose: Intune will provide customer the ability to run their Powershell scripts on the enrolled windows 10 Azure Active Directory joined devices. The script can be run once or periodically.

## Requirements: 

### Scopes Required
- DeviceManagementManagedDevices.ReadWrite.All

### A client AppID/Secret or Device Code Authentication
[Docs.Microsoft.com - List deviceManagementScripts](https://docs.microsoft.com/en-us/graph/api/intune-shared-devicemanagementscript-list?view=graph-rest-beta)
        
    Important: Microsoft Graph APIs under the /beta version are subject to change; production use is not supported.
    Note: The Microsoft Graph API for Intune requires an active Intune license for the tenant.

## Set Variables and Import required modules and credentials 

In [35]:
## Change these if required
$resource = "https://graph.microsoft.com" ## resource Dont change since we want to query MDATP REST API Resource
$grant_type = "client_credentials" ## This is using a device_code flow (supports: client_credentials, device_code)
$apiVersion = "beta"  ## Identify the version of the API ( api or beta )

## Dont really need to change these unless necessary
$VerbosePreference = "ignore"   ## Set to 'continue to get verbose output'
## This will run a script to import the required modules and credentials stored in the "credential" file
$scriptPath = "~/Notebooks/PowerShell/custom-modules/import.ps1"; . $scriptPath  # this will run the script


CommandType     Name                                               Version    Source
-----------     ----                                               -------    ------
Function        Get-DecryptedStringUsingMasterPassword             0.0        pp-core
Function        Get-EncodedString                                  0.0        pp-core
Function        Get-EncryptedStringUsingMasterPassword             0.0        pp-core
Function        Get-HashOfString                                   0.0        pp-core
Function        Get-MasterPassword                                 0.0        pp-core
Function        Get-StoredCredentials                              0.0        pp-core
Function        Invoke-CheckCredentials                            0.0        pp-core
Function        New-StoreCredentials                               0.0        pp-core
Function        Get-MSGraphAuthToken                               0.0        graph
Function        Invoke-GraphAuthentication               

## Authenticate with Graph API and get access Token for MDATP Resource

In [36]:
## Get a token if authenticates properly. 

$authParams = @{
    tenantId = $credentials.tenantId
    client_id = $credentials.client_id
    client_secret = $credentials.client_secret
    resource = $resource  ## resource Dont change since we want to query MDATP REST API Resource
    grant_type = $grant_type  ## This is using a device_code flow (supports: client_credentials, device_code)
    #scope = "https://api.securitycenter.microsoft.com/Machine.ReadWrite.All" ## set the scope
};

## this will authenticate with Graph API and token will be stored in <tokenResponse> variable.
$token = (Invoke-GraphAuthentication -authParams $authParams)

## This is the authentication header for REST API Call
if ($token.access_token -ne $null) {
    $authHeader = @{ 
        'Content-Type' = 'application/json'
        Accept = 'application/json'
        Authorization = "Bearer $($token.access_token)" 
    }
}; $authHeader


Authentication Parameters detected
Personal Access Token (PAT) grant_type

----------------------------------------------------------------------------
Authentiating with Microsoft Graph API using a Personal Access Token (PAT)
https://docs.microsoft.com/en-us/azure/storage/common/storage-auth-aad-app
----------------------------------------------------------------------------
Requesting Token at https://login.microsoftonline.com/e4194ef7-b5d7-4db1-b7e9-a73af406f276/oauth2/token


Received Token!
Connected and Access Token received and will expire 1612936082

Name                           Value
----                           -----
Accept                         application/json
Content-Type                   application/json
Authorization                  Bearer eyJ0eXAiOiJKV1QiLCJub25jZSI6ImZtM2JNTWFLUVdocTZWdGZtSnpJWjRIYnNDNFZhRXpmOUdqNTJaVWI1djQiLCJhbGciOiJSUzI1NiIsIng1dCI6Im5PbzNaRHJPRFhFSzFqS1doWHNsSFJfS1hFZyIsImtp…



## Get list of available scripts

In [37]:
## Set the path you want to query
$endpoint = "deviceManagement/deviceManagementScripts"
$method = "GET"

## The following commands will query the graph API and output the first 5 machines in table format
$uri = "$($resource)/$apiVersion/$endpoint"; $uri
$Response = (Invoke-RestMethod -Method $method -Uri $uri -Headers $authHeader -body $body -ErrorAction Stop)

## Filter columns 
$filter = @('id','displayName','description','fileName')
$Response.value | Select-Object $filter | FT

https://graph.microsoft.com/beta/deviceManagement/deviceManagementScripts

id                                   displayName  description            fileName
--                                   -----------  -----------            --------
4838170d-a36e-47c0-aa4c-6ae9aa971d90 test script2 This is a test script. test2.ps1
5456b4d2-cce1-4c4e-93c4-b4078423b620 test script  This is a test script. test.ps1
b34e96e5-9580-4228-8319-6636ecc4b3ec test script2 This is a test script. test2.ps1



## Create a sample script

In [30]:
$script = @"

## This is a test script
echo 'test' > c:\test.txt

"@

$encodedString = Get-encodedString $script

Encoded String: CiMjIFRoaXMgaXMgYSB0ZXN0IHNjcmlwdAplY2hvICd0ZXN0JyA+IGM6XHRlc3QudHh0Cg==


## Create a test powershell script

In [31]:
## Set the path you want to query
$endpoint = "deviceManagement/deviceManagementScripts"
$method = "POST"

$body = @"
{
  "@odata.type": "#microsoft.graph.deviceManagementScript",
  "displayName": "test script2",
  "description": "This is a test script.",
  "scriptContent": "$encodedString",
  "runAsAccount": "user",
  "enforceSignatureCheck": true,
  "fileName": "test2.ps1",
  "runAs32Bit": true
}
"@

## The following commands will query the graph API and output the first 5 machines in table format
$uri = "$($resource)/beta/$endpoint"; $uri
$Response = (Invoke-RestMethod -Method $method -Uri $uri -Headers $authHeader -body $body -ErrorAction Stop)
$Response


https://graph.microsoft.com/beta/deviceManagement/deviceManagementScripts

@odata.context        : https://graph.microsoft.com/beta/$metadata#deviceManagement/deviceManagementScripts/$entity
enforceSignatureCheck : True
runAs32Bit            : True
id                    : b34e96e5-9580-4228-8319-6636ecc4b3ec
displayName           : test script2
description           : This is a test script.
scriptContent         : CiMjIFRoaXMgaXMgYSB0ZXN0IHNjcmlwdAplY2hvICd0ZXN0JyA+IGM6XHRlc3QudHh0Cg==
createdDateTime       : 2/10/2021 3:57:39 AM
lastModifiedDateTime  : 2/10/2021 3:57:39 AM
runAsAccount          : user
fileName              : test2.ps1
roleScopeTagIds       : {0}




## Assign script to a group of devices
- https://docs.microsoft.com/en-us/graph/api/intune-shared-devicemanagementscript-assign?view=graph-rest-beta

In [ ]:
## Set the path you want to query
$endpoint = "deviceManagement/deviceManagementScripts"
$method = "POST"

$body = @"
{
  "@odata.type": "#microsoft.graph.deviceManagementScript",
  "displayName": "test script2",
  "description": "This is a test script.",
  "scriptContent": "$encodedString",
  "runAsAccount": "user",
  "enforceSignatureCheck": true,
  "fileName": "test2.ps1",
  "runAs32Bit": true
}
"@

## The following commands will query the graph API and output the first 5 machines in table format
$uri = "$($resource)/beta/$endpoint"; $uri
$Response = (Invoke-RestMethod -Method $method -Uri $uri -Headers $authHeader -body $body -ErrorAction Stop)
$Response